In [3]:
from distutils.log import error
from open_crypto import runner
from matplotlib import pyplot as plt
import sqlite3 
import pandas as pd
import datetime

#params 
depths_ranges = [50,100,150,200]

#create database
depth_db = pd.DataFrame(columns=['exchange', 'mid price', 'spread', 'depth','depth units','depth units USD', 'time'])

In [6]:
#fetch data from exchanges
def check_dbstatus():
    if con.total_changes == 0:
        return(print("DB connection established"))
    else:
        return(print("failed to establish connection to database"))

#fetch data from exchanges
runner.run('amkt.yaml')
# Read sqlite query results into a pandas DataFrame

con = sqlite3.connect("amkt_data.db")

check_dbstatus()

orderbook = pd.read_sql_query('SELECT * FROM order_books_view;', con)
prices = pd.read_sql_query('SELECT exchange, MAX(price) AS price FROM trades_view GROUP BY exchange;', con)
# Verify that result of SQL query is stored in the dataframe

#get the time of the snapshot
cur_time = datetime.datetime.now() 

con.close()

def calculate_depth(database, exchange, range): #depths are calculated in BPS
    range = range / 100 #convert to kBPS (1000BPS = 1KBPS)
    #get midprice of the last range of trades
    price = prices.loc[prices['exchange'] == exchange, 'price'].item()

    #calculate depth boundaries
    depth_asks = price * (1 + range)
    depth_bids = price - (price * range)

    #get trades within the depth boundaries
    trades = database.loc[(database['bids_price'] > depth_bids) & (database['asks_price'] < depth_asks)]
    trades = trades.assign(**{'price': trades['bids_price'].fillna(trades['asks_price'])})
    

    #calculate depth
    depth = trades['price'].std()

    return depth

def calculate_spread(database, exchange):
    #get midprice of the last range of trades
    bidmaxprice = database.loc[database['exchange'] == exchange, 'bids_price'].max()
    askminprice = database.loc[database['exchange'] == exchange, 'asks_price'].min()

    #calculate spread
    spread = bidmaxprice - askminprice

    return spread

#calculate orderbook depth for each exchange and write to a new database
for exchange in orderbook['exchange'].unique():
    for depth_val in depths_ranges:
        df = pd.DataFrame()
        depth_units = calculate_depth(orderbook, exchange, depth_val)
        spread = calculate_spread(orderbook, exchange)
        depth_units_USD = depth_units * prices.loc[prices['exchange'] == exchange, 'price'].item()
        row = [exchange, prices.loc[prices['exchange'] == exchange, 'price'].item(), spread, depth_val, depth_units, depth_units_USD, cur_time] 
        depth_db.loc[len(depth_db)] = row

depth_db

Requesting 2 exchange(s) for job: orderbook.                                    
Requesting 2 exchange(s) for job: tickers.
Requesting data... 50.00 % /                                                    Pair-ID 336 - Coinbase: 17276 tuple(s)
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
Pair-ID 336 - Coinbase: 420 tuple(s)                                            
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
DB connection established


,exchange,mid price,spread,depth,depth units,depth units USD,time
0,COINBASE,1531.64,19.11,50,126.632684,193955.683947,2022-07-25 19:28:28.449374
1,COINBASE,1531.64,19.11,100,211.333512,323686.860996,2022-07-25 19:28:28.449374
2,COINBASE,1531.64,19.11,150,385.756312,590839.797322,2022-07-25 19:28:28.449374
3,COINBASE,1531.64,19.11,200,433.741743,664336.203172,2022-07-25 19:28:28.449374
4,FTX,1530.10,19.10,50,126.382826,193378.362068,2022-07-25 19:28:28.449374
5,FTX,1530.10,19.10,100,210.951090,322776.263075,2022-07-25 19:28:28.449374
6,FTX,1530.10,19.10,150,384.393213,588160.055464,2022-07-25 19:28:28.449374
7,FTX,1530.10,19.10,200,433.741743,663668.240888,2022-07-25 19:28:28.449374
8,COINBASE,1531.64,19.11,50,126.651773,193984.920870,2022-07-25 19:30:54.915674
9,COINBASE,1531.64,19.11,100,211.352436,323715.845734,2022-07-25 19:30:54.915674


In [5]:
depth_db

,exchange,mid price,spread,depth,depth units,depth units USD,time
0,COINBASE,1531.64,19.11,50,126.632684,193955.683947,2022-07-25 19:28:28.449374
1,COINBASE,1531.64,19.11,100,211.333512,323686.860996,2022-07-25 19:28:28.449374
2,COINBASE,1531.64,19.11,150,385.756312,590839.797322,2022-07-25 19:28:28.449374
3,COINBASE,1531.64,19.11,200,433.741743,664336.203172,2022-07-25 19:28:28.449374
4,FTX,1530.10,19.10,50,126.382826,193378.362068,2022-07-25 19:28:28.449374
5,FTX,1530.10,19.10,100,210.951090,322776.263075,2022-07-25 19:28:28.449374
6,FTX,1530.10,19.10,150,384.393213,588160.055464,2022-07-25 19:28:28.449374
7,FTX,1530.10,19.10,200,433.741743,663668.240888,2022-07-25 19:28:28.449374
